<h1><b>PROJECT HELPING HANDS Model Training</b></h1>

Members:
1.   Renmar Balana................Leading Developer / Project Manager
2.   Xania Shane Oropesa......Full Stack Developer
3.   Hewey James Lita...........Tester / Front End Developer
4.   Jan Lance Borrero...........Frontend Developer, UI/UX designer
5.   Jiggy Brondial..................Data Gathering / Cleaning

*BSCS 3-A (2022-2023)*

---

## **ROADMAP TO CREATING AN OBJECT DETECTION TFLITE MODEL**


* Collect the dataset of images and label them to get their xml files.

* Install the TensorFlow Object Detection API.

* Generate the TFRecord files required for training. (need generate_tfrecord.py script and csv files for this)

* Edit the model pipeline config file and download the pre-trained model checkpoint.

* Train and evaluate the model.

* Export and convert the model into TFlite(TensorFlow Lite) format.

* Deploy the TFlite model on Android / iOS / IoT devices.




# **0) Checking Prerequisites**

At the time of testing, this notebook works with tensorflow version 2.12, and Python version 3.9.16. 
If the current version of python does not work, try changing to alternative python versions by running the code below.

In [ ]:
!python --version
!sudo update-alternatives --config python3

In [ ]:
!sudo apt install python3-pip

# **1) Import Libraries**

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)

# **2) Create *`customTF2`*, *`training`* and *`data`* folders in your google drive**
Create a folder named ***customTF2*** in your google drive.

Create another folder named ***training*** inside the ***customTF2*** folder
(***training*** folder is where the checkpoints will be saved during training)

Create another folder named ***data*** inside the ***customTF2*** folder.

# **3) Create and upload your image files and xml files.**
 Create a folder named ***images*** for your custom dataset images and create another folder named ***annotations*** for its corresponding xml files.
 
 Next, create their zip files and upload them to the ***customTF2*** folder in your drive.

 
 (Make sure all the image files have extension as ".jpg" only.
 Other formats like ".png" , ".jpeg" or even ".JPG" will give errors since the generate_tfrecord and xml_to_csv scripts here have only ".jpg" in them)

## Collect Images Dataset and label them to get their PASCAL_VOC XML annotations
For Datasets, you can check out my Dataset Sources at the bottom of this article in the credits section. You can use any software for labeling like the labelImg tool.

Read this [article](https://viso.ai/computer-vision/labelimg-for-image-annotation/) to know more about collecting datasets and labeling process.

# **4) Upload the *`generate_tfrecord.py`* file to the *`customTF2`* folder on your drive.**


You can find the generate_tfrecord.py file [here](https://github.com/techzizou/Train-Object-Detection-Model-TF-2.x)

# **5) Mount drive and link your folder**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# **6) Clone the tensorflow models git repository & Install TensorFlow Object Detection API**

In [ ]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# **7) Test the model builder**

In [ ]:
# testing the model builder to make sure all the required componenets are present.
!python object_detection/builders/model_builder_tf2_test.py

# **8) Navigate to /mydrive/customTF2/data/ and Unzip the *images.zip* and *annotations.zip* files into the *data* folder**

In [ ]:
%cd /mydrive/customTF2/data/

# unzip the datasets and their contents so that they are now in /mydrive/customTF2/data/ folder
!unzip /mydrive/customTF2/images.zip -d .
!unzip /mydrive/customTF2/annotations.zip -d .

# **9) Create test_labels & train_labels**
Current working directory is /mydrive/customTF2/data/

Divide annotations into test_labels(20%) and train_labels(80%).

In [ ]:
%cd /mydrive/customTF2/data/

#creating two dir for training and testing
!mkdir test_labels train_labels

# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 271/1355 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -271 | xargs -I{} mv {} test_labels/


# Moves the rest of the labels ( 1084 labels ) to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/